In [ ]:
#| default_exp utils

# Utilities
> Various utilities

In [ ]:
#| export
from pathlib import Path
from netCDF4 import Dataset
from fastcore.test import test_eq
import fastcore.all as fc
import pandas as pd
import numpy as np
import requests
from shapely import MultiPoint
from operator import attrgetter

from marisco.configs import species_lut_path, sediments_lut_path

import jellyfish as jf
from collections.abc import Callable

## Callbacks

In [ ]:
#| export
class Callback(): order = 0

In [ ]:
#| export
def run_cbs(cbs, obj=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        if cb.__doc__: obj.logs.append(cb.__doc__)
        cb(obj)

In [ ]:
#| export
class Transformer():
    def __init__(self, dfs, cbs=None): 
        fc.store_attr()
        self.logs = []
        
    def callback(self):
        run_cbs(self.cbs, self)
        
    def __call__(self):
        self.callback()
        return self.dfs

## Validation

In [ ]:
#| export
def has_valid_varname(
    var_names:list, # variable (nuclide) names
    cdl_path:str, # Path to MARIS CDL file (point of truth)
):
    "Check that proposed variable names are in MARIS CDL"
    has_valid = True
    with Dataset(cdl_path) as nc:
        grp = nc.groups[list(nc.groups.keys())[0]] # get any group
        for name in var_names:
            if name not in grp.variables.keys():
                has_valid = False
                print(f'"{name}" variable name not found in MARIS CDL')
    
    return has_valid

In [ ]:
VARNAMES = ['lat', 'lon']
test_eq(has_valid_varname(VARNAMES, './files/nc/maris-cdl.nc'), True)

In [ ]:
VARNAMES = ['ba140_invalid', 'ba140_dl']
test_eq(has_valid_varname(VARNAMES, './files/nc/maris-cdl.nc'), False)

"ba140_invalid" variable name not found in MARIS CDL


## Geoprocessing

In [ ]:
#| export
def get_bbox(df,
             coord_cols=('lon', 'lat')
            ):
    x, y = coord_cols        
    arr = [(row[x], row[y]) for _, row in df.iterrows()]
    return MultiPoint(arr).envelope

In [ ]:
df = pd.DataFrame({'lon': np.linspace(-10, 5, 20), 'lat':  np.linspace(40, 50, 20)})
bbox = get_bbox(df);

In [ ]:
# To get `lon_min`, `lon_max`, `lat_min`, `lat_max`
bbox.bounds

(-10.0, 40.0, 5.0, 50.0)

In [ ]:
# And its Well-Know Text representation
bbox.wkt

'POLYGON ((-10 40, 5 40, 5 50, -10 50, -10 40))'

In [ ]:
# If unique (lon, lat)
df = pd.DataFrame({'lon': [0, 0], 'lat':  [1, 1]})
bbox = get_bbox(df);

In [ ]:
bbox.bounds

(0.0, 1.0, 0.0, 1.0)

## Downloaders

In [ ]:
#| export
def download_files_in_folder(owner:str, 
                             repo:str, 
                             src_dir:str, 
                             dest_dir:str
                             ):
    "Make a GET request to the GitHub API to get the contents of the folder"
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{src_dir}"
    response = requests.get(url)

    if response.status_code == 200:
        contents = response.json()

        # Iterate over the files and download them
        for item in contents:
            if item["type"] == "file":
                fname = item["name"]
                download_file(owner, repo, src_dir, dest_dir, fname)
    else:
        print(f"Error: {response.status_code}")

def download_file(owner, repo, src_dir, dest_dir, fname):
    # Make a GET request to the GitHub API to get the raw file contents
    url = f"https://raw.githubusercontent.com/{owner}/{repo}/master/{src_dir}/{fname}"
    response = requests.get(url)

    if response.status_code == 200:
        # Save the file locally
        with open(Path(dest_dir) / fname, "wb") as file:
            file.write(response.content)
        print(f"{fname} downloaded successfully.")
    else:
        print(f"Error: {response.status_code}")

## WorRMS
The [World Register of Marine Species (WorMS)](https://www.marinespecies.org) is an authoritative classification and catalogue of marine names. It provides a REST API (among others) allowing to "fuzzy" match any species name you might encounter in marine data sources names againt their own database. There are several types of matches as described [here](https://www.marinespecies.org/tutorial_taxonmatch.php).

In [ ]:
#| export
def match_worms(
    name:str # Name of species to look up in WoRMS
    ):
    "Lookup `name` in WoRMS (fuzzy match)"
    url = 'https://www.marinespecies.org/rest/AphiaRecordsByMatchNames'
    params = {
        'scientificnames[]': [name],
        'marine_only': 'true'
    }
    headers = {
        'accept': 'application/json'
    }
    
    response = requests.get(url, params=params, headers=headers)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return -1

For instance:

In [ ]:
#|eval: false
match_worms('Aristeus antennatus')

[[{'AphiaID': 107083,
   'url': 'https://www.marinespecies.org/aphia.php?p=taxdetails&id=107083',
   'scientificname': 'Aristeus antennatus',
   'authority': '(Risso, 1816)',
   'status': 'accepted',
   'unacceptreason': None,
   'taxonRankID': 220,
   'rank': 'Species',
   'valid_AphiaID': 107083,
   'valid_name': 'Aristeus antennatus',
   'valid_authority': '(Risso, 1816)',
   'parentNameUsageID': 106807,
   'kingdom': 'Animalia',
   'phylum': 'Arthropoda',
   'class': 'Malacostraca',
   'order': 'Decapoda',
   'family': 'Aristeidae',
   'genus': 'Aristeus',
   'citation': 'DecaNet eds. (2024). DecaNet. Aristeus antennatus (Risso, 1816). Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=107083 on 2024-06-05',
   'lsid': 'urn:lsid:marinespecies.org:taxname:107083',
   'isMarine': 1,
   'isBrackish': 0,
   'isFreshwater': 0,
   'isTerrestrial': 0,
   'isExtinct': 0,
   'match_type': 'exact',
   'modified': '2022-08-24T09:48:1

In [ ]:
match_worms('Plankton')

[[{'AphiaID': 747442,
   'url': 'https://www.marinespecies.org/aphia.php?p=taxdetails&id=747442',
   'scientificname': 'Plankayon',
   'authority': 'Wagner, 1880',
   'status': 'unaccepted',
   'unacceptreason': 'invalid as based on nomen nudum',
   'taxonRankID': 180,
   'rank': 'Genus',
   'valid_AphiaID': 117064,
   'valid_name': 'Plotocnide',
   'valid_authority': 'Wagner, 1885',
   'parentNameUsageID': 22766,
   'kingdom': 'Animalia',
   'phylum': 'Cnidaria',
   'class': 'Hydrozoa',
   'order': 'Anthoathecata',
   'family': None,
   'genus': 'Plankayon',
   'citation': 'Schuchert, P. (2024). World Hydrozoa Database. Plankayon Wagner, 1880. Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=747442 on 2024-06-05',
   'lsid': 'urn:lsid:marinespecies.org:taxname:747442',
   'isMarine': 1,
   'isBrackish': 0,
   'isFreshwater': 0,
   'isTerrestrial': 0,
   'isExtinct': 0,
   'match_type': 'near_2',
   'modified': '2014-01-29T1

## Marisco look-up table fuzzy matching
Using https://jamesturk.github.io/jellyfish fuzzy matching distance metrics.


### Biota species

In [ ]:
#| export
def match_maris_species(
    lut_path:str, # Path to MARIS species authoritative species look-up table
    name:str, # Name of species to look up 
    col_lookup:str='species', # Name of the column where the character strings match
    dist_fn:Callable=jf.levenshtein_distance, # Jellyfish distance to use
    coi:list=['species_id', 'species', 'Taxonname', 'TaxonDBID'], # Columns of interest to display
    nresults:int=10 # Maximum number of results to return
    ):
    "Lookup `name` in MARIS master species lookup table"
    df = pd.read_excel(lut_path)
    df = df.dropna(subset=col_lookup)
    df = df.astype({'species_id':'int'})
    results = []
    for _, row in df.iterrows():
        score = dist_fn(name.lower(), row[col_lookup].lower())
        result = row[coi].to_dict()
        result['score'] = score
        results.append(result)
    return pd.DataFrame(results).sort_values(by='score', ascending=True)[:nresults]

In [ ]:
# species_lut_path()
lut_fname = '../files/lut/dbo_species_cleaned.xlsx'
match_maris_species(lut_fname, 'PLANKTON')

,species_id,species,Taxonname,TaxonDBID,score
281,280,Plankton,Plankton,Q25367,0
696,695,Zooplankton,Zooplankton,Q842627,3
633,632,Palaemon,Palaemon,Q1269791,4
160,159,Neuston,Neuston,Q1552545,5
697,696,Phytoplankton,Phytoplankton,Q184755,5
812,811,Chanos,Chanos,Q15114233,5
220,219,Pagurus,Pagurus,Q2480027,6
867,866,Terapon,Terapon,Q1978136,6
1233,1232,Praunus,Praunus,Q3860707,6
1019,1018,Therapon,Therapon,Q27988534,6


### Sediment types

In [ ]:
#| export
def match_maris_sediment(
    name:str, # Name of sediment to look up 
    col_lookup:str='sedtype', # Name of the column where the character strings match
    dist_fn:Callable=jf.levenshtein_distance, # Jellyfish distance to use
    coi:list=['sedtype_id', 'sedtype'], # Columns of interest to display
    nresults:int=10 # Maxiumn number of results to return
    ):
    "Lookup `sedtype` in MARIS master sediment lookup table"
    df = pd.read_excel(sediments_lut_path())
    df = df.dropna(subset=col_lookup)
    df = df.astype({'sedtype_id':'int'})
    results = []
    for _, row in df.iterrows():
        score = dist_fn(name.lower(), row[col_lookup].lower())
        result = row[coi].to_dict()
        result['score'] = score
        results.append(result)
    return pd.DataFrame(results).sort_values(by='score', ascending=True)[:nresults]

In [ ]:
match_maris_sediment('GLACIAL')

,sedtype_id,sedtype,score
25,25,Glacial,0
2,2,Gravel,4
1,1,Clay,5
50,50,Glacial clay,5
3,3,Marsh,6
14,14,Sludge,6
6,6,Sand,6
12,12,Silt,6
51,51,Soft clay,7
15,15,Turf,7
